# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

misschien handige code om te gebruiken:

%pwd (de data moet in je workdirectery staan)


In [1]:
import numpy as np
import pandas as pd
import datetime

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

In [2]:
dfProjectTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Project tasks')
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Rooms')
dfHiring = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Hiring')

Het schoonmaken van de cel, hierdoor kunnen we waardes makkelijker worden teruggevonden dan NaN

In [3]:
dfProjectTasks = dfProjectTasks.fillna(0)
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfCrew = dfCrew.fillna(0)
dfRooms = dfRooms.fillna(1)

In [4]:
dfProjectTasks.replace("x", 1, inplace=True)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfCrew.replace("x", 1, inplace=True)


**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken (hoe lang is de pauze die moet worden genomen? >> half uur)
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' maar 1 keer per sprint worden gedaan of kunnen meerdere personen er een volgen? zoveel als je wilt 
- crewleden moet te samen aan de skills komen voor de opdracht.
- moeten de mensen die skill 1 nog niet hebben deze ook leren? hoeven niet te worden geleerd
- kan een crewlid eerst een uur aan het ene project werken, dit niet afronden en dan nog aan een ander project? >> bij project tasks moet het aan eengesloten zijn, bij additional tasks hoeft dit niet
- moet iedereen tegelijk aan het project werken >> moet!

Stel we maken een soort kalender, startend op 1 januari 2020. 
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- de taak
- persoon die het uitvoerd
- begintijd 
- eindtijd
- Tijdsduur?
- Of het crewlid is ingehuurd (T/F)
- Kosten zodra het crewlid is ingehuurd
- Of het een werkdag is (T/F)

In [5]:
data = pd.date_range("2020-01-01", periods=4*10*7, freq='D')

columns = ['Datum', 'Task','Crewlid', 'begintijd', 'eindtijd', 'tijdsduur', 'ingehuurd', 'kosten', 'werkdag']

dfKalender = pd.DataFrame(columns=columns)
dfKalender['Datum'] = data
dfKalender = dfKalender.fillna(0)
dfKalender.replace("0", False, inplace=True)


Het toevoegen of een dag een werkdag is. Er wordt in het weekend niet gewerkt dus deze dagen zullen afvallen.

In [6]:
dfKalender['werkdag'] = dfKalender['Datum'].apply(lambda Datum: Datum.isoweekday() in range (1,6))

Er moet een functie zijn die kan worden aangeroepen om te kijken of een taak door een bepaald persoon kan worden uitgevoerd.

Hier moeten de eisen van de opdracht worden meegegeven en een persoon. Vervolgens krijg je een boolean terug of het kan.

In [7]:
# een paar standaard variabele om de code leesbaar te maken.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

functie waarbij de lijst van ReqSkill moet worden ingevuld van de Task en de skills van een persoon. De functie geeft een boolean terug of deze taak door deze persoon kan worden gedaan.

voorbeeld: <br>
OpdrachtUitvoeren(dfProjectTasks.iloc[0,rangeSkills],dfCrew.iloc[0,rangeSkills]) 
<br>uitkomst: False

In [8]:
def OpdrachtUitvoerenPersoonBool(eisen, persoon):
    for i in rangeSkills:
        result = (int(eisen[i-1]) <= 
                  int(persoon[i-1]))
        if result == False:
            break
    return result

de functie hieronder voegt de skills van personen samen en geeft de gezamelijke skills terug.

voorbeeld: PersonenSamenvoegen(dfCrew.iloc[0,rangeSkills],dfCrew.iloc[2,rangeSkills])
<br>
uitkomst: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

In [9]:
def PersonenSamenvoegen(persoon1, persoon2, persoon3 = None, persoon4 = None, persoon5 = None, persoon6 = None):
    combinedSkill = []
    for i in rangeSkills:
        combinedSkill.append(int(1) if persoon1[i-1]+persoon2[i-1] > 0 else int(0)) 
    return combinedSkill

In [ ]:
# het selecteren van alleen de tasks van sprint 1
dfProjectTasks = dfProjectTasks[dfProjectTasks['Task'] != 0]
dfSprint1 = dfProjectTasks[dfProjectTasks['Task'].str.contains('S1')]


In [54]:
dfCrewCombined = []
for i in range(0, len(dfCrew)-1): # combinaties met twee mensen
    for j in range(0, len(dfCrew)-1):
        if i==j:
            break
        t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills])
        t.append(i)
        t.append(j)
        t.append(20)
        t.append(20)
        t.append(20)
        t.append(20)
        
        dfCrewCombined.append(t)

for i in range(0, len(dfCrew)-1): # combinaties met drie personen
    for j in range(0, len(dfCrew)-1):
        if i==j:
            break
        for k in range(0, len(dfCrew)-1):
            if i==k or k==j:
                break
        
            t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills], dfCrew.iloc[k,rangeSkills])
            t.append(i)
            t.append(j)
            t.append(k)
            t.append(20)
            t.append(20)
            t.append(20)
            
            dfCrewCombined.append(t)
            
for i in range(0, len(dfCrew)-1): # combinaties met vier personen
    for j in range(0, len(dfCrew)-1):
        if i==j:
            break
        for k in range(0, len(dfCrew)-1):
            if i==k or k==j:
                break
            for l in range(0, len(dfCrew)-1):
                if l == k or l == i or l == j :
                    break
                t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
                                        dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills])
                t.append(i)
                t.append(j)
                t.append(k)
                t.append(l)
                t.append(20)
                t.append(20)
                dfCrewCombined.append(t)

for i in range(0, len(dfCrew)-1): # combinaties met vijf personen
    for j in range(0, len(dfCrew)-1):
        if i==j:
            break
        for k in range(0, len(dfCrew)-1):
            if i==k or k==j:
                break
            for l in range(0, len(dfCrew)-1):
                if l == k or l == i or l == j :
                    break
                for m in range(0, len(dfCrew)-1):
                    if m==l or m==k or m==j or m==i:
                        break
                    t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
                                            dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
                                            dfCrew.iloc[m,rangeSkills])
                    t.append(i)
                    t.append(j)
                    t.append(k)
                    t.append(l)
                    t.append(m)
                    t.append(20)
                    dfCrewCombined.append(t)
                    
for i in range(0, len(dfCrew)-1): # combinaties met vijf personen
    for j in range(0, len(dfCrew)-1):
        if i==j:
            break
        for k in range(0, len(dfCrew)-1):
            if i==k or k==j:
                break
            for l in range(0, len(dfCrew)-1):
                if l == k or l == i or l == j :
                    break
                for m in range(0, len(dfCrew)-1):
                    if m==l or m==k or m==j or m==i:
                        break
                    for n in range(0, len(dfCrew)-1):
                        if n==m or n==l or n==k or n==j or n==i:
                            break
                        t = PersonenSamenvoegen(dfCrew.iloc[i,rangeSkills], dfCrew.iloc[j,rangeSkills],
                                                dfCrew.iloc[k,rangeSkills], dfCrew.iloc[l,rangeSkills], 
                                                dfCrew.iloc[m,rangeSkills], dfCrew.iloc[n,rangeSkills])
                        t.append(i)
                        t.append(j)
                        t.append(k)
                        t.append(l)
                        t.append(m)
                        t.append(n)
                        dfCrewCombined.append(t)

                    
       

In [55]:
ar = np.array(dfCrewCombined, dtype=np.int16)
df = pd.DataFrame(ar, columns= ['Skill1', 'Skill2', 'Skill3', 'Skill4', 'Skill5', 'Skill6', 'Skill7', 'Skill8', 
                                'Skill9', 'Skill10', 'Skill11', 'Skill12', 'Skill13', 'persoon1', 'persoon2', 
                                'persoon3', 'persoon4', 'persoon5', 'persoon6'])

In [56]:
df

,Skill1,Skill2,Skill3,Skill4,Skill5,Skill6,Skill7,Skill8,Skill9,Skill10,Skill11,Skill12,Skill13,persoon1,persoon2,persoon3,persoon4,persoon5,persoon6
0,1,0,0,0,0,1,1,1,1,1,0,1,0,1,0,20,20,20,20
1,1,1,1,1,1,1,0,1,1,1,1,1,1,2,0,20,20,20,20
2,1,1,1,1,1,1,1,1,1,0,1,1,1,2,1,20,20,20,20
3,1,0,0,0,0,1,0,1,0,1,1,1,1,3,0,20,20,20,20
4,1,0,0,0,0,1,1,1,1,1,1,1,1,3,1,20,20,20,20
5,1,1,1,1,1,1,0,1,1,1,1,0,1,3,2,20,20,20,20
6,0,0,0,0,1,0,1,1,0,1,1,1,1,4,0,20,20,20,20
7,1,0,0,0,1,1,1,1,1,0,1,1,1,4,1,20,20,20,20
8,1,1,1,1,1,1,1,1,1,0,1,0,1,4,2,20,20,20,20
9,1,0,0,0,1,1,1,1,0,1,1,0,1,4,3,20,20,20,20
